In [1]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.16-py3-none-any.whl size=110685 sha256=6a85d4dfae5f329642696d37fb15498440646d50785d13bd4f6bb22353e08d2b
  Stored in directory: /root/.cache/pip/wheels/d2/ed/a5/da3a0cfb13373d1ace41cafa4f2467d858c55c52473ba72799
Successfully built kaggle


In [13]:
# !cp ../kaggle.json /root/.kaggle/kaggle.json
!

nn_from_scrath_replication.ipynb  titanic


In [14]:
# Copying over from JH notebook
import os
from pathlib import Path
import pandas as pd
import numpy as np

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle: path = Path('../input/titanic')
else:
    path = Path('titanic')
    if not path.exists():
        import zipfile,kaggle
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

ls: cannot access 'replication/titanic': No such file or directory


In [39]:
import pandas as pd
import numpy as np
train_df = pd.read_csv(path/'train.csv')

In [67]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [60]:
# TODO: Check which ones JH included
cat_vars = ['Pclass', 'Sex', 'Embarked']
num_vars = ['Age', 'SibSp', 'Parch', 'Fare']

In [56]:
num_df = train_df[num_vars].fillna(train_df[num_vars].mode().iloc[0])
num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)

# Going through an easier approach rather than typical norm
num_df = num_df / num_df.max()
num_df

,Age,SibSp,Parch,Fare
0,0.2750,0.125,0.000000,0.317521
1,0.4750,0.125,0.000000,0.683873
2,0.3250,0.000,0.000000,0.331789
3,0.4375,0.125,0.000000,0.636672
4,0.4375,0.000,0.000000,0.334298
...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118
887,0.2375,0.000,0.000000,0.545154
888,0.3000,0.125,0.333333,0.505672
889,0.3250,0.000,0.000000,0.545154


In [66]:
train_df[cat_vars].describe(include='all')
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [68]:
train_df.Pclass.unique()

array([3, 1, 2])

In [71]:
# Curious how in-depth you should conside N/As
cat_df = pd.get_dummies(train_df[cat_vars], columns=cat_vars)
cat_df

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1,0,1,0,0,1
1,1,0,0,1,0,1,0,0
2,0,0,1,1,0,0,0,1
3,1,0,0,1,0,0,0,1
4,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...
886,0,1,0,0,1,0,0,1
887,1,0,0,1,0,0,0,1
888,0,0,1,1,0,0,0,1
889,1,0,0,0,1,1,0,0


In [75]:
# Might need to normalize after creating the dummy vars?
proc_train_df = num_df.join(cat_df)

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.2750,0.125,0.000000,0.317521,0,0,1,0,1,0,0,1
1,0.4750,0.125,0.000000,0.683873,1,0,0,1,0,1,0,0
2,0.3250,0.000,0.000000,0.331789,0,0,1,1,0,0,0,1
3,0.4375,0.125,0.000000,0.636672,1,0,0,1,0,0,0,1
4,0.4375,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118,0,1,0,0,1,0,0,1
887,0.2375,0.000,0.000000,0.545154,1,0,0,1,0,0,0,1
888,0.3000,0.125,0.333333,0.505672,0,0,1,1,0,0,0,1
889,0.3250,0.000,0.000000,0.545154,1,0,0,0,1,1,0,0


NameError: name 'df' is not defined

In [100]:
def process_df(path):
    df = pd.read_csv(path)
    df = df.fillna(df.mode().iloc[0])
    
    
    # TODO: Check which ones JH included
    cat_vars = ['Pclass', 'Sex', 'Embarked']
    num_vars = ['Age', 'SibSp', 'Parch', 'Fare']
    
    
    num_df = df[num_vars]
    num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)
    # Going for an easier approach rather than typical norm
    num_df = num_df / num_df.max()

    cat_df = pd.get_dummies(df[cat_vars], columns=cat_vars)
    cat_df['const'] = 1
    
    return num_df.join(cat_df)

In [101]:
train_df = process_df(path/'train.csv')
train_df

/tmp/ipykernel_62/2267849073.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)


,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.2750,0.125,0.000000,0.317521,0,0,1,0,1,0,0,1,1
1,0.4750,0.125,0.000000,0.683873,1,0,0,1,0,1,0,0,1
2,0.3250,0.000,0.000000,0.331789,0,0,1,1,0,0,0,1,1
3,0.4375,0.125,0.000000,0.636672,1,0,0,1,0,0,0,1,1
4,0.4375,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118,0,1,0,0,1,0,0,1,1
887,0.2375,0.000,0.000000,0.545154,1,0,0,1,0,0,0,1,1
888,0.3000,0.125,0.333333,0.505672,0,0,1,1,0,0,0,1,1
889,0.3250,0.000,0.000000,0.545154,1,0,0,0,1,1,0,0,1


In [102]:
train_df.isna().sum()

Age           0
SibSp         0
Parch         0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
const         0
dtype: int64

In [103]:
test_df = process_df(path/'test.csv')
test_df

/tmp/ipykernel_62/2267849073.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)


,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.453947,0.000,0.000000,0.329840,0,0,1,0,1,0,1,0,1
1,0.618421,0.125,0.000000,0.311896,0,0,1,1,0,0,0,1,1
2,0.815789,0.000,0.000000,0.363976,0,1,0,0,1,0,1,0,1
3,0.355263,0.000,0.000000,0.346051,0,0,1,0,1,0,0,1,1
4,0.289474,0.125,0.111111,0.402083,0,0,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.276316,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
414,0.513158,0.000,0.000000,0.751796,1,0,0,1,0,1,0,0,1
415,0.506579,0.000,0.000000,0.317521,0,0,1,0,1,0,0,1,1
416,0.276316,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1


In [104]:
train_df.isna().sum()

Age           0
SibSp         0
Parch         0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
const         0
dtype: int64

In [ ]:
# Great success! We can move into building out model!

In [97]:
import torch

In [109]:
weights = torch.rand(train_df.shape[1]) - 0.5
weights

tensor([ 0.2510, -0.0758,  0.0975, -0.2264, -0.0464, -0.4071,  0.0476, -0.4948,
        -0.1597,  0.0943,  0.4170, -0.3862,  0.1851])

In [120]:
tensor_df = torch.tensor(train_df.values)
tensor_df

tensor([[0.2750, 0.1250, 0.0000,  ..., 0.0000, 1.0000, 1.0000],
        [0.4750, 0.1250, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.3250, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 1.0000],
        ...,
        [0.3000, 0.1250, 0.3333,  ..., 0.0000, 1.0000, 1.0000],
        [0.3250, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.4000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000]],
       dtype=torch.float64)

In [124]:
result = (tensor_df * weights)
result

tensor([[ 0.0690, -0.0095,  0.0000,  ...,  0.0000, -0.3862,  0.1851],
        [ 0.1192, -0.0095,  0.0000,  ...,  0.0000, -0.0000,  0.1851],
        [ 0.0816, -0.0000,  0.0000,  ...,  0.0000, -0.3862,  0.1851],
        ...,
        [ 0.0753, -0.0095,  0.0325,  ...,  0.0000, -0.3862,  0.1851],
        [ 0.0816, -0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.1851],
        [ 0.1004, -0.0000,  0.0000,  ...,  0.4170, -0.0000,  0.1851]],
       dtype=torch.float64)

In [123]:
result.shape

torch.Size([891, 13])

In [119]:
train_df.shape

(891, 13)

In [131]:
def sigmoid(x):
    return 1 / (1 + 1 / torch.exp(-x))

In [135]:
torch_implementation_sigmoid = torch.sigmoid(result.sum(axis=1))
my_implementation_sigmoid = sigmoid(result.sum(axis=1))

In [141]:
# Awesome!
(torch_implementation_sigmoid == torch_implementation_sigmoid).sum()

tensor(891)

In [ ]:
# TODO: Implement actual training